In [1]:
import pandas as pd
import numpy as np
import joblib 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest

# загрузим данные
df = pd.read_csv(r'G:/Python_source/fake_job_postings.csv',sep=',')
# объединим несколько полей объявлений для каждой записи в одной строке
df['text'] = df[['title', 'location', 'department', 'company_profile', 'description', 'requirements', 'benefits']].apply(lambda x: ' '.join(str(i) for i in x if not pd.isnull(i)), axis=1)

# выберем только необходимые столбцы
df = df[['text', 'fraudulent']]

# разделим данные на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['fraudulent'], test_size=0.2, random_state=42)

# создадим объект TfidfVectorizer и обучим его на тренировочных данных
tfidf = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1,2))
X_train_tfidf = tfidf.fit_transform(X_train)

# применим обученный трансформатор к тестовым данным
X_test_tfidf = tfidf.transform(X_test)

# добавляем SMOTE для балансировки классов
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)



In [2]:
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
# определяем диапазоны значений параметров
param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [3, 5, 7, 10]
}
smote = SMOTE(random_state=42)

# применим обученный трансформатор к тестовым данным
X_test_tfidf = tfidf.transform(X_test)

# создадим объект StandardScaler и обучим его на тренировочных данных
scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(X_train_tfidf)

# применим обученный трансформатор к тестовым данным
X_test_scaled = scaler.transform(X_test_tfidf)

# добавляем SMOTE для балансировки классов
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)
X_train_scaled_resampled = scaler.fit_transform(X_train_resampled)
# создаем экземпляр модели 

xgb_model = XGBClassifier()

# инициализируем объект GridSearchCV
grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='f1')

# инициализируем модель XGBoost
xgb_model = XGBClassifier(n_estimators=100, max_depth=5)

# обучим модель на тренировочных данных
xgb_model.fit(X_train_scaled_resampled, y_train_resampled)

# сделаем предсказания на тестовых данных
xgb_y_pred_proba = xgb_model.predict_proba(X_test_scaled)[:, 1]

# выберем порог, максимизирующий F1-меру
f1_scores_xgb = []
for threshold in np.arange(0.1, 1, 0.1):
    xgb_y_pred_thresholded = (xgb_y_pred_proba > threshold).astype(int)
    f1_scores_xgb.append(f1_score(y_test, xgb_y_pred_thresholded))
    
best_threshold_xgb = np.arange(0.1, 1, 0.1)[np.argmax(f1_scores_xgb)]

#превращаем вероятности в классы с использованием лучшего порога
xgb_y_pred = (xgb_y_pred_proba > best_threshold_xgb).astype(int)

In [ ]:
#from xgboost import XGBClassifier

# инициализируем модель XGBoost
xgb_model = XGBClassifier(n_estimators=500, max_depth=10)

# обучим модель на тренировочных данных
xgb_model.fit(X_train_scaled_resampled, y_train_resampled)

# сделаем предсказания на тестовых данных
xgb_y_pred_proba = xgb_model.predict_proba(X_test_scaled)[:, 1]

# выберем порог, максимизирующий F1-меру
f1_scores_xgb = []
for threshold in np.arange(0.1, 1, 0.1):
    xgb_y_pred_thresholded = (xgb_y_pred_proba > threshold).astype(int)
    f1_scores_xgb.append(f1_score(y_test, xgb_y_pred_thresholded))
    
best_threshold_xgb = np.arange(0.1, 1, 0.1)[np.argmax(f1_scores_xgb)]

#превращаем вероятности в классы с использованием лучшего порога
xgb_y_pred = (xgb_y_pred_proba > best_threshold_xgb).astype(int)

#оценим качество модели
xgb_accuracy = accuracy_score(y_test, xgb_y_pred) 
xgb_recall = recall_score(y_test, xgb_y_pred) 
xgb_f1 = f1_score(y_test, xgb_y_pred)

In [3]:
def train_model(X_train, y_train):
    # создадим объект TfidfVectorizer и обучим его на тренировочных данных
    tfidf = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1,2))
    X_train_tfidf = tfidf.fit_transform(X_train)

    # добавляем SMOTE для балансировки классов
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

    # создадим объект SelectKBest для отбора признаков
    kbest = SelectKBest(k=1000)

    # применим метод отбора признаков к тренировочным данным
    X_train_tfidf_kbest = kbest.fit_transform(X_train_resampled, y_train_resampled)

    # создадим модель логистической регрессии и обучим ее на отобранных признаках
    lr_model = LogisticRegression() 
    lr_model.fit(X_train_tfidf_kbest, y_train_resampled)

    # создаем объект RandomForestClassifier
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

    # обучаем модель на отобранных признаках
    rf_model.fit(X_train_tfidf_kbest, y_train_resampled)

    # создаем объект DecisionTreeClassifier
    dt_model = DecisionTreeClassifier(random_state=42)

    # обучаем модель на отобранных признаках
    dt_model.fit(X_train_tfidf_kbest, y_train_resampled)

    # сохраняем модели в файлы с помощью joblib
    joblib.dump(tfidf, 'tfidf.joblib')
    joblib.dump(smote, 'smote.joblib')
    joblib.dump(kbest, 'kbest.joblib')
    joblib.dump(lr_model, 'lr_model.joblib')
    joblib.dump(rf_model, 'rf_model.joblib')
    joblib.dump(dt_model, 'dt_model.joblib')
    return lr_model
def predict(new_data):
    # загрузим сохраненные модели из файлов
    tfidf = joblib.load('tfidf.joblib')
    smote = joblib.load('smote.joblib')
    kbest = joblib.load('kbest.joblib')
    # get trained logistic regression model
    lr_model = train_model(X_train_resampled, y_train_resampled)
    lr_model = joblib.load('lr_model.joblib')
    rf_model = joblib.load('rf_model.joblib')
    dt_model = joblib.load('dt_model.joblib')

    # преобразуем новые данные с помощью TfidfVectorizer и SelectKBest
    new_data_tfidf = tfidf.transform(new_data)
    new_data_tfidf_kbest = kbest.transform(new_data_tfidf)

    # получение предсказаний класса для каждой модели
    y_pred_lr = lr_model.predict(new_data_tfidf_kbest)
    y_pred_rf = rf_model.predict(new_data_tfidf_kbest)
    y_pred_dt = dt_model.predict(new_data_tfidf_kbest)

    # возвращаем предсказания классов для каждой модели
    return y_pred_lr, y_pred_rf, y_pred_dt


In [22]:
import tkinter as tk

# Создание окна приложения
root = tk.Tk()
root.title('Модель для прогнозирования мошеннических объявлений')
root.geometry("1000x600")
lr_model = joblib.load('lr_model.joblib')
# Создание элементов пользовательского интерфейса
label = tk.Label(root, text='Введите данные для прогнозирования')
text_box = tk.Text(root, height=20, width=80)
button_paste = tk.Button(root, text='Вставить из буфера обмена')
button_clear = tk.Button(root, text='Очистить текст')
button_predict = tk.Button(root, text='Предсказать')
label_model_status = tk.Label(root, text='Модель LogisticRegression загружена')
label_result = tk.Label(root, text='Результат:')

# Определение расположения элементов на окне
label.grid(row=1, column=0, padx=10, pady=10)
text_box.grid(row=2, column=0, padx=10, pady=10)
button_paste.grid(row=3, column=0, padx=10, pady=10)
button_clear.grid(row=3, column=1, padx=10, pady=10)
button_predict.grid(row=4, column=0, padx=10, pady=10)
label_result.grid(row=5, column=0, padx=10, pady=10)
label_model_status.grid(row=0, column=1, sticky='ne')

# Обработчики событий кнопок
def paste_from_clipboard():
    text_box.delete('1.0', tk.END)
    text_box.insert(tk.END, root.clipboard_get())

def clear_text_box():
    text_box.delete('1.0', tk.END)

def predict():
    new_data = [text_box.get('1.0', tk.END)]
    new_data_tfidf = tfidf.transform(new_data) 
    new_data_tfidf_kbest = kbest.transform(new_data_tfidf)
    y_pred_new = lr_model.predict(new_data_tfidf_kbest)
    y_pred_new = lr_model.predict(new_data_tfidf_kbest)
    if y_pred_new == 0:
        text="Это объявление относится к категории реальных вакансий"
    else:
        text="Это объявление относится к категории поддельных вакансий"
    label_result.config(text='Результат: ' +text )

# Привязка обработчиков к кнопкам
button_paste.config(command=paste_from_clipboard)
button_clear.config(command=clear_text_box)
button_predict.config(command=predict)

# Запуск главного цикла приложения
root.mainloop()

In [ ]:
# Поддельные объявления
#Power Plant Energy Edison International and Refined Resources have partnered up in an effort
#Gg gg 39000-56000,As Part Time Administrative Assistantyou will be responsible
#Admin Assistant Receptionist Anapa 23040-28800 Newly established company seeks outgoing friendly people person attitude

In [ ]:
#Реальные объявления
#Teacher Volgograd 500-600 high We help teachers get safe amp secure jobs abroad :) Play with kids, get paid 